In [1]:
import os
from subprocess import check_call
import yaml

import intake

import operators as ops
import utils
import funnel

In [2]:
esm_collection_json = 'data/campaign-cesm2-cmip6-timeseries.json'
catalog = intake.open_esm_datastore(esm_collection_json)
catalog

,unique
Unnamed: 0,312503
experiment,13
case,33
component,6
stream,22
variable,2641
date_range,596
member_id,12
path,312503
ctrl_branch_year,18


In [3]:
with open('datasets.yml') as fid:
    datasets = yaml.safe_load(fid)

In [4]:
data_dir = f'/glade/scratch/{os.environ["USER"]}/cesm2-marbl-data'
if True:
    check_call(['rm', '-fr', data_dir])
os.makedirs(data_dir, exist_ok=True)        

In [5]:
with open('_config_calc.yml') as fid:
    config_dict = yaml.load(fid, Loader=yaml.Loader)

_collections = config_dict['data_collections']
_collections

{'epoch_mean': {'experiment': {'historical': ['1990', '2014'],
   'SSP5-RCP8.5': ['2086', '2100'],
   'SSP1-2.6': ['2086', '2100'],
   'SSP2-4.5': ['2086', '2100'],
   'SSP3-7.0': ['2086', '2100'],
   'SSP5-8.5': ['2086', '2100']}},
 'global_mean_timeseries_ann': None,
 'global_integral_timeseries_ann': None}

In [6]:
cluster, client = utils.get_ClusterClient()
cluster.scale(12) #adapt(minimum_jobs=0, maximum_jobs=24)
client

Client Scheduler: tcp://10.12.206.63:34640 Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [10]:
clobber = False

cdf_kwargs_stream = {
    'pop.h': dict(chunks={'time': 12}, decode_coords=False),
    'pop.h.ecosys.nyear1': dict(chunks={'time': 1}, decode_coords=False),
}

dfs = {}; paths = {}
for collection_type, exp_dict in datasets.items():
    
    for experiment, variable_list in exp_dict.items():
            
            cat = catalog.search(
                experiment=experiment, 
                variable=variable_list, 
                component='ocn')
            
            for v in variable_list:
                
                # subset catalog
                cat_sub = cat.search(variable=v)
                keys = list(cat_sub.keys())
                if not keys:
                    print(f'missing: {collection_type}, {experiment}, {v}')
                    continue
                
                elif len(keys) > 1:                
                    #assert len(keys) == 1, f'expecting single key, found: {keys}'
                    print(f'{v} -- expecting single key, found: {keys}')
                    continue
                    
                # get info
                key = keys[0]
                key_info = funnel.core._intake_esm_get_keys_info(cat_sub)[key]                       
                stream = key_info['stream']
                
                date_range = cat_sub.df.date_range.to_list()
                date_range = '-'.join([date_range[0].split('-')[0], date_range[-1].split('-')[-1]]) 
                
                # set operations
                sel_dict = dict()
                if collection_type == 'timeseries':
                    pass 
                
                elif collection_type == 'epoch_mean':
                    time_range = _collections['epoch_mean']['experiment'][experiment]                    
                    sel_dict = dict(time=slice(time_range[0], time_range[1]))                        
                    
                    if stream == 'pop.h':
                        date_range = '-'.join([time_range[0]+'01', time_range[-1]+'12'])
                    
                    elif stream == 'pop.h.ecosys.nyear1':
                        date_range = '-'.join([time_range[0], time_range[-1]])                    
                
                # name this dataset
                name = f"cesm2-cmip6.{experiment}.{stream}.{v}.{date_range}"
                file_out = f'{data_dir}/{name}.zarr'
                
                # to write or not to write? if not, move on
                if os.path.exists(file_out) and not clobber:
                    continue 

                # get the dataset
                dsets = cat_sub.to_dataset_dict(cdf_kwargs=cdf_kwargs_stream[stream])
                _, ds = dsets.popitem()
                
                # center the time axix
                ds = ops.center_time(ds)

                # apply operations                
                if sel_dict:
                    ds = ds.sel(**sel_dict)[[v]]
                
                # write
                ds.to_zarr(file_out, mode='w')

SST -- expecting single key, found: ['ocn.historical.pop.h', 'ocn.historical.pop.h.nday1']

--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream'



--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream'



--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream'



--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream'



--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream'


KeyError: 'pop.h.nday1'

In [ ]:
# data_dir = f'/glade/scratch/{os.environ["USER"]}/cesm2-marbl-data'
# os.makedirs(data_dir, exist_ok=True)

# data_vol_GB = {}
# for key, paths in paths.items():
#     data_vol_GB[key] = 0.
#     for path in paths[key]:
#         data_vol_GB[key] += os.stat(path).st_size / 1024**3
#         check_call(['ln', '-sf', path, data_dir+'/.'])
        
#     print(f'Data volume {key}: {data_vol_GB[key]:0.2f} GB')    

In [8]:
v


'SST'